In [1]:
#from lib.project_5 import load_data_from_database, make_data_dict, general_model, general_transformer

# Step 1 - Benchmarking

**NOTE: EACH OF THESE SHOULD BE WRITTEN SOLELY WITH REGARD TO STEP 1 - BENCHMARKING**

### Domain and Data

**TODO:** Write a simple statement about the domain of your problem and the dataset upon which you will be working. 
Prepared for the Neural Information Processing Symposium 2003 Feature Extraction Workshop

http://clopinet.com/isabelle/Projects/NIPS2003

### Data 

MADELON is an artificial dataset, which was part of the NIPS 2003 feature selection challenge. This is a two-class classification problem with continuous input variables. The difficulty is that the problem is multivariate and highly non-linear.

MADELON is an artificial dataset containing data points grouped in 32 clusters placed on the vertices of a five dimensional hypercube and randomly labeled +1 or -1. The five dimensions constitute 5 informative features. 15 linear combinations of those features were added to form a set of 20 (redundant) informative features. Based on those 20 features one must separate the examples into the 2 classes (corresponding to the +-1 labels). We added a number of distractor feature called 'probes' having no predictive power. The order of the features and patterns were randomized. 

### Problem Statement

**TODO:** Write a simple problem statement with regard to benchmarking your work only.
The NIPS 2003 challenge in feature selection is to find feature selection algorithms that significantly outperform methods using all features in performing a binary classification task.

### Solution Statement

**TODO:** Write a simple solution statement with regard to benchmarking your work only.
We will develop a binary classification model and attempt to augment its performance using automatic feature selection techniques.

### Metric

**TODO**: Write a statement about the metric you will be using. This section is global as it will be the metric you will use throughout this project. Provide a brief justification for choosing this metric.

We will examine all of the metrics that we have used so far - Accuracy, Precision, Recall, and F1-Score, as well as look at a Confusion Matrix. For comparing models, we will use Accuracy.

### Benchmark

**TODO**: Write a statement explaining that this is the process by which you identify a benchmark for your project.
We will use as a benchmark our mean accuracy across five random train test splits using a K Nearest Neighbors model with an optimal value for number of `n_neighbors`. This model had a 73.8% accuracy.


## Implementation

Implement the following code pipeline using the functions you write in `lib/project_5.py`.

<img src="assets/benchmarking.png" width="600px">

In [1]:
from sqlalchemy import create_engine
import pandas as pd
#from os import chdir; chdir('..')
from os import chdir; chdir('./lib')
from sklearn.preprocessing import StandardScaler
from project_5 import load_data_from_database, make_data_dict,general_transformer, general_model
from sklearn import linear_model
#from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier


In [41]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [38]:
!pwd

/src/dsi/dsi-workspace/project-05/lib


In [ ]:
pd.read_sql()

In [3]:
madelon_df = load_data_from_database()
madelon_df.head(5)

,index,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_491,feat_492,feat_493,feat_494,feat_495,feat_496,feat_497,feat_498,feat_499,label
0,0,485,477,537,479,452,471,491,476,475,...,481,477,485,511,485,481,479,475,496,-1
1,1,483,458,460,487,587,475,526,479,485,...,478,487,338,513,486,483,492,510,517,-1
2,2,487,542,499,468,448,471,442,478,480,...,481,492,650,506,501,480,489,499,498,-1
3,3,480,491,510,485,495,472,417,474,502,...,480,474,572,454,469,475,482,494,461,1
4,4,484,502,528,489,466,481,402,478,487,...,479,452,435,486,508,481,504,495,511,1


In [4]:
madelon_df.shape

(2000, 502)

In [5]:
madelon_df.describe().T

,count,mean,std,min,25%,50%,75%,max
index,2000.0,999.5000,577.494589,0.0,499.75,999.5,1499.25,1999.0
feat_000,2000.0,481.7225,6.421769,462.0,477.00,482.0,486.00,503.0
feat_001,2000.0,483.4525,30.186294,381.0,464.00,483.0,503.00,600.0
feat_002,2000.0,510.1660,38.899165,370.0,485.00,510.5,536.00,654.0
feat_003,2000.0,483.3845,9.059895,453.0,477.00,483.0,490.00,519.0
feat_004,2000.0,501.6125,41.389418,371.0,475.00,500.0,528.00,688.0
feat_005,2000.0,479.2590,6.795956,459.0,475.00,479.0,484.00,505.0
feat_006,2000.0,480.1095,40.575925,334.0,452.75,480.0,506.25,611.0
feat_007,2000.0,476.5650,1.384461,471.0,476.00,477.0,477.00,481.0
feat_008,2000.0,486.7935,15.043836,430.0,477.00,487.0,496.25,536.0


In [6]:
madelon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 502 entries, index to label
dtypes: int64(502)
memory usage: 7.7 MB


**Check for null values in all columns.**

In [7]:
[madelon_df[pd.isnull(madelon_df[col])].count() for col in madelon_df.columns] 

[index       0
 feat_000    0
 feat_001    0
 feat_002    0
 feat_003    0
 feat_004    0
 feat_005    0
 feat_006    0
 feat_007    0
 feat_008    0
 feat_009    0
 feat_010    0
 feat_011    0
 feat_012    0
 feat_013    0
 feat_014    0
 feat_015    0
 feat_016    0
 feat_017    0
 feat_018    0
 feat_019    0
 feat_020    0
 feat_021    0
 feat_022    0
 feat_023    0
 feat_024    0
 feat_025    0
 feat_026    0
 feat_027    0
 feat_028    0
            ..
 feat_471    0
 feat_472    0
 feat_473    0
 feat_474    0
 feat_475    0
 feat_476    0
 feat_477    0
 feat_478    0
 feat_479    0
 feat_480    0
 feat_481    0
 feat_482    0
 feat_483    0
 feat_484    0
 feat_485    0
 feat_486    0
 feat_487    0
 feat_488    0
 feat_489    0
 feat_490    0
 feat_491    0
 feat_492    0
 feat_493    0
 feat_494    0
 feat_495    0
 feat_496    0
 feat_497    0
 feat_498    0
 feat_499    0
 label       0
 dtype: int64, index       0
 feat_000    0
 feat_001    0
 feat_002    0
 feat_003  

In [8]:
madelon_df["label"].unique()

array([-1,  1])

** Calculate the Baseline Accuracy **

**`baseline_accuracy = majority_class_N / total_population`**

In [9]:
madelon_df.groupby(by="label")["index"].count()

label
-1    1000
 1    1000
Name: index, dtype: int64

In [10]:
baseline_accuracy = 1000*1./madelon_df["label"].count()
baseline_accuracy

0.5

**Baseline Accuracy is Set to 50%**

## Define X and Y

In [11]:
# looking at the mean of Index column and its name it seems it is a counter and does not contribute towards 
# the study, so we will remove that column from study
del madelon_df["index"]
# Define X and y
y = madelon_df["label"]
X = madelon_df.drop("label",axis=1)

In [12]:
data_dict= make_data_dict(X,y,0.5,random_state=42)

In [13]:
data_dict['X_train'].head()

,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_490,feat_491,feat_492,feat_493,feat_494,feat_495,feat_496,feat_497,feat_498,feat_499
440,473,470,511,480,555,480,475,476,493,485,...,482,478,469,505,487,529,481,482,524,491
573,481,503,527,479,529,479,447,475,484,469,...,490,481,474,672,461,537,469,496,572,476
946,477,445,499,497,434,475,481,478,500,482,...,523,477,482,788,488,479,475,495,501,474
997,491,468,495,484,548,479,533,478,465,464,...,494,474,504,473,489,483,476,494,565,457
503,469,489,494,492,445,484,484,477,474,481,...,481,483,510,613,486,528,485,504,510,511


In [14]:
data_dict = general_transformer(StandardScaler(),data_dict)
data_dict

{'X_test': array([[ 1.12156758, -0.51026631,  0.33005212, ..., -0.52713324,
         -0.07137678,  0.47371054],
        [-0.3047583 , -0.67637944,  1.08365312, ..., -0.01450019,
         -0.07137678,  0.47371054],
        [ 0.17068366, -1.34083196, -0.64962918, ...,  1.66986554,
          0.79543799, -0.22014492],
        ..., 
        [ 0.17068366, -0.77604732, -0.09698845, ..., -1.47916605,
          0.30785468,  0.04968776],
        [ 0.80460628,  2.34687951,  0.48077232, ...,  1.96279872,
         -0.28808047,  0.82063828],
        [ 0.48764497, -0.04514955, -2.15683118, ..., -0.74683312,
         -2.02171001,  1.01337591]]),
 'X_train': array([[-1.41412287, -0.44382106,  0.02861172, ..., -0.08773348,
          0.44329449,  0.04968776],
        [-0.14627764,  0.65252559,  0.43053225, ...,  0.93753262,
          1.74351665, -0.52852513],
        [-0.78020026, -1.2743867 , -0.27282868, ...,  0.86429932,
         -0.17972863, -0.60562018],
        ..., 
        [-0.14627764,  0.287076

In [15]:
data_dict.keys()

['X_test', 'processes', 'X_train', 'y_train', 'y_test']

In [16]:
transformer = data_dict["processes"][0]

In [17]:
data_dict.keys()

['X_test', 'processes', 'X_train', 'y_train', 'y_test']

**Below model is EDA Purposes only in which we are going to leverage sklearn:**

1. We leverage standardized and transformed train data from tranformed process. 
2. Fit the train feature and target set 
4. Create a Logistic Regression with l2 i.e. Ridge with large C value to minimize regularization.  
6. Evalute the score.
7. Review the coefficients
8. Create a mapping of coefficient and features
9. Write analysis on my finding

In [18]:
# run preliminary logistic regression for EDA purpose
model = linear_model.LogisticRegression(penalty = 'l2', C=1E10)
data_dict = general_model(model, data_dict)
#model.fit(data_dict["X_train"], data_dict["y_train"])
data_dict.keys()

['X_test',
 'processes',
 'X_train',
 'test_score',
 'train_score',
 'y_train',
 'y_test']

In [20]:
model = data_dict["processes"][1]

In [34]:
print "Train Score:", data_dict["train_score"], "Test Score:", data_dict["test_score"]

Train Score: 1.0 Test Score: 0.544


In [37]:
feature_coef = []
for i,k in enumerate(X.columns):
    feature_coef.append([k, model.coef_[0][i]])
df_coef=pd.DataFrame(feature_coef)
df_coef.columns = ["feature","coef_"]

df_coef.sort_values(["coef_"],ascending=False)


,feature,coef_
475,feat_475,34.224080
453,feat_453,29.932129
46,feat_046,29.267086
317,feat_317,24.742018
136,feat_136,22.464534
251,feat_251,20.894102
445,feat_445,20.791725
151,feat_151,20.580548
455,feat_455,20.216010
116,feat_116,19.432239


With the naive LogisticRegression it appears that model worked very well on train set however performed poorly on the test data set. ** The train score was perfectly 1.0 and test data score dcid poorly 0.544 **. This suggest we need to further tune the model with more penaly or even performing LogisticRegression using Lasso method. 